In [1]:
import tensorflow as tf
from tensorflow.keras import optimizers, Sequential,layers,metrics,datasets
from tensorflow import keras

In [47]:
(x,y),(x_val,y_val) = datasets.fashion_mnist.load_data()

In [9]:
def preprocess(x,y):
    x = 2*tf.cast(x,dtype= tf.float32) / 255. -1.
    y = tf.cast(y,dtype= tf.int32)   
    return x,y

In [48]:
y = tf.one_hot(y,depth=10)


In [50]:
y_val = tf.squeeze(y_val)

In [52]:
y_val = tf.one_hot(y_val,depth=10)

In [56]:
batchsz = 128
db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(10000).batch(batchsz)
db_test = tf.data.Dataset.from_tensor_slices((x_val,y_val))
db_test = db_test.map(preprocess).batch(batchsz)

In [57]:
next(iter(db_test))[1].shape

TensorShape([128, 10])

In [63]:
# 自定义层
class MyDense(layers.Layer):
    def __init__(self,in_dim,out_dim):
        super(MyDense,self).__init__()
        self.kernel = self.add_weight('w',[in_dim,out_dim])
#         self.bias = self.add_weight('b',[out_dim])
    def call(self,inputs,training = None):
        out = inputs@self.kernel
        return out

In [65]:
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.fc1 = MyDense(28*28,256)
        self.fc2 = MyDense(256,128)
        self.fc3 = MyDense(128,64)
        self.fc4 = MyDense(64,32)
        self.fc5 = MyDense(32,10)
    def call(self,inputs,training = None):
        x = tf.reshape(inputs,[-1,28*28])
        x = self.fc1(x)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.relu(x)
        x = self.fc3(x)
        x = tf.nn.relu(x)
        x = self.fc4(x)
        x = tf.nn.relu(x)
        x = self.fc5(x)
    
        return x

In [66]:
network = MyModel()
network.compile(optimizer=optimizers.Adam(learning_rate= 0.01),
                loss=tf.losses.CategoricalCrossentropy(from_logits= True),
                metrics=['accuracy']
)

In [67]:
network.fit(db,epochs= 5,validation_data=db_test,validation_freq=2)
network.evaluate(db_test)

Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.6083 - accuracy: 0.7843
Epoch 2/5
469/469 [==============================] - 3s 6ms/step - loss: 0.4287 - accuracy: 0.8466 - val_loss: 0.4331 - val_accuracy: 0.8431
Epoch 3/5
469/469 [==============================] - 3s 5ms/step - loss: 0.3953 - accuracy: 0.8577
Epoch 4/5
469/469 [==============================] - 3s 6ms/step - loss: 0.3739 - accuracy: 0.8672 - val_loss: 0.4370 - val_accuracy: 0.8471
Epoch 5/5
79/79 [==============================] - 0s 3ms/step - loss: 0.4285 - accuracy: 0.8516


[0.42854413390159607, 0.8515999913215637]